In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import pickle
import os
from ttkthemes import ThemedTk
from PIL import Image, ImageTk
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

class StudentPredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Student Prediction App")
        self.root.geometry("600x500")
        self.root.configure(background="#f0f0f0")

        self.file1_path = None
        self.file2_path = None
        self.model_path = None

        self.create_widgets()

    def create_widgets(self):
        img_url = "https://ros-spravka.ru/upload/iblock/94f/izhevsk_state_technical_university-logo.png"
        img_data = self.get_image_from_url(img_url)
        img_data = img_data.resize((100, 100), Image.BICUBIC)
        self.photo = ImageTk.PhotoImage(img_data)
        
        frame = tk.Frame(self.root, bg="#f0f0f0")
        frame.pack(pady=20, padx=20, fill="both", expand=True)
        
        image_label = tk.Label(frame, image=self.photo, bg="#f0f0f0")
        image_label.grid(row=0, column=1, padx=10, pady=10, sticky="n")

        # Frame для загрузки файла 1
        file1_frame = tk.Frame(frame, bg="#f0f0f0")
        file1_frame.grid(row=1, column=0, pady=10, sticky="w")

        tk.Label(file1_frame, text="Выберите excel файл с оценками студентов:", bg="#f0f0f0").pack(anchor="w")
        self.file1_entry = tk.Entry(file1_frame, width=50)
        self.file1_entry.pack(side="left", padx=(0, 10))
        self.file1_button = tk.Button(file1_frame, text="Обзор", command=self.load_file1)
        self.file1_button.pack(side="left")

        # Frame для загрузки файла 2
        file2_frame = tk.Frame(frame, bg="#f0f0f0")
        file2_frame.grid(row=2, column=0, pady=10, sticky="w")

        tk.Label(file2_frame, text="Выберите excel файл с информацией о студентах:", bg="#f0f0f0").pack(anchor="w")
        self.file2_entry = tk.Entry(file2_frame, width=50)
        self.file2_entry.pack(side="left", padx=(0, 10))
        self.file2_button = tk.Button(file2_frame, text="Обзор", command=self.load_file2)
        self.file2_button.pack(side="left")

        # Frame для загрузки модели
        model_frame = tk.Frame(frame, bg="#f0f0f0")
        model_frame.grid(row=3, column=0, pady=10, sticky="w")

        tk.Label(model_frame, text="Выберите модель:", bg="#f0f0f0").pack(anchor="w")
        self.model_entry = tk.Entry(model_frame, width=50)
        self.model_entry.pack(side="left", padx=(0, 10))
        self.model_button = tk.Button(model_frame, text="Обзор", command=self.load_model)
        self.model_button.pack(side="left")

        # Кнопка для выполнения прогноза
        self.predict_button = tk.Button(frame, text="Сделать прогноз", command=self.make_prediction)
        self.predict_button.grid(row=4, column=0, pady=20, sticky="w")

    def load_file1(self):
        self.file1_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if self.file1_path:
            self.file1_entry.delete(0, tk.END)
            self.file1_entry.insert(0, self.file1_path)
            messagebox.showinfo("Info", f"Файл 1 загружен: {self.file1_path}")

    def load_file2(self):
        self.file2_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if self.file2_path:
            self.file2_entry.delete(0, tk.END)
            self.file2_entry.insert(0, self.file2_path)
            messagebox.showinfo("Info", f"Файл 2 загружен: {self.file2_path}")

    def load_model(self):
        self.model_path = filedialog.askopenfilename(filetypes=[("Pickle files", "*.pkl")])
        if self.model_path:
            self.model_entry.delete(0, tk.END)
            self.model_entry.insert(0, self.model_path)
            messagebox.showinfo("Info", f"Модель загружена: {self.model_path}")

    def process_data(self, file1_path, file2_path):
        try:
            df_grades = pd.read_excel(file1_path)
            df_persons_2017 = pd.read_excel(file2_path, sheet_name='Персоны', skiprows=[0, 1])
            df_students_2017 = pd.read_excel(file2_path, sheet_name='Обучающиеся', skiprows=[0, 1, 2, 3, 4, 5, 6])
            
            df_grades = df_grades.drop(columns=["student_status", "student_compensation_type", "student_program", "staff_fio", "subject", "date"])
            
            df_persons_2017 = df_persons_2017.drop(columns=["№", "Пол", "Дата рождения", "Льготы", "Иностранный язык", "Законченное образ. учреждение (осн. док.)", "Полученное образование (осн. док.)"])
            
            df_students_2017 = df_students_2017.drop(columns=["№", "Дата рождения", "Полученное образование (док. студ.)", "Законченное образ. учреждение (док. студ.)"])
            
            df_test = pd.merge(
                df_persons_2017, df_students_2017,
                left_on='Документ о полученном образовании (осн. док.)',
                right_on='Документ о полученном образовании (док. студ.)',
                how='inner')
            
            df_test.dropna(subset=['Документ о полученном образовании (док. студ.)'], inplace=True)
            
            pivot_df = df_grades.pivot_table(index=['student_number'], columns='mark', aggfunc='size', fill_value=0).reset_index()
            
            df_test['Зачетная книжка'] = df_test['Зачетная книжка'].astype("str") 
            pivot_df['student_number'] = pivot_df['student_number'].astype("str")
            
            df_test = df_test[df_test['Зачетная книжка'].str.isdigit()]
            pivot_df = pivot_df[pivot_df['student_number'].str.isdigit()]
            
            df_merged = pd.merge(
                df_test, pivot_df,
                left_on='Зачетная книжка',
                right_on='student_number',
                how='inner')
            
            df_merged = df_merged.dropna(subset=['Документ о полученном образовании (осн. док.)', 'Документ о полученном образовании (док. студ.)'])
            
            df_col = df_merged['student_number']
            
            df_merged = df_merged.drop(columns=['Документ о полученном образовании (осн. док.)',
                                                'Ср. балл док-та об образовании (осн. док.)',
                                                'Личный номер',
                                                'Зачетная книжка',
                                                'Группа',
                                                'Направленность',
                                                'Нормативный срок',
                                                'Организация ЦП',
                                                'Год приема',
                                                'student_number',
                                                'Курс',
                                                'Документ о полученном образовании (док. студ.)',
                                                'Перечень',
                                                'Направление подготовки (специальность)'])
            
            df_merged = df_merged.rename(columns={"Иностранное гражданство": "Гражданство"})
            df_merged.fillna({'Гражданство': 'Россия', 'Город': 'г. Ижевск (Респ. Удмуртская)'}, inplace=True)
            df_merged['Ср. балл док-та об образовании (док. студ.)'] = df_merged['Ср. балл док-та об образовании (док. студ.)'].fillna(df_merged['Ср. балл док-та об образовании (док. студ.)'].mean())
            
            df_merged['Status'] = np.where(df_merged['Состояние'] != 'отчислен', 1, 0)
            df_merged.drop(columns=['Состояние'], inplace=True)
            
            df_merged['Город_Категория'] = np.where(df_merged['Город'] == 'г. Ижевск (Респ. Удмуртская)', 1, 0)
            df_merged.drop(columns=['Город'], inplace=True)
            
            df_merged['Гражданство_Категория'] = np.where(df_merged['Гражданство'] == 'Россия', 1, 0)
            df_merged.drop(columns=['Гражданство'], inplace=True)
            
            mean_value = df_merged['Ср. балл док-та об образовании (док. студ.)'].mean()
            df_merged['Ср. балл док-та об образовании (док. студ.)'].replace(0, mean_value, inplace=True)
            
            X = df_merged.drop('Status', axis=1)
            y = df_merged['Status']
            
            # Определение категориальных и числовых столбцов
            categorical_features = ['Пол', 'Формирующее подр.', 'Присваиваемая квалификация', 'Вид затрат', 'Форма освоения', 'Условие освоения', 'Целевик']
            numeric_features = [col for col in X.columns if col not in categorical_features]
            
            # Создание ColumnTransformer для автоматического преобразования категориальных данных в числовые значения
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', StandardScaler(), numeric_features),
                    ('cat', OrdinalEncoder(), categorical_features)
                ])
            
            X_processed = preprocessor.fit_transform(X)
            return X_processed, df_col

        except Exception as e:
            messagebox.showerror("Error", f"Ошибка при обработке данных: {e}")
            return None, None

    def make_prediction(self):
        if not self.file1_path or not self.file2_path or not self.model_path:
            messagebox.showwarning("Warning", "Пожалуйста, загрузите все файлы.")
            return

        X_processed, student_numbers = self.process_data(self.file1_path, self.file2_path)
        if X_processed is None:
            return

        try:
            with open(self.model_path, 'rb') as f:
                model = pickle.load(f)
        except Exception as e:
            messagebox.showerror("Error", f"Ошибка при загрузке модели: {e}")
            return

        try:
            predictions = model.predict(X_processed)
        except Exception as e:
            messagebox.showerror("Error", f"Ошибка при предсказании: {e}")
            return

        try:
            output_df = pd.DataFrame({
                'student_number': student_numbers,
                'Prediction': predictions
            })
            output_path = os.path.join(os.path.dirname(self.file1_path), "Прогноз.xlsx")
            output_df.to_excel(output_path, index=False)
            messagebox.showinfo("Info", f"Прогноз сохранен в: {output_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Ошибка при сохранении прогноза: {e}")

    @staticmethod
    def get_image_from_url(url):
        from io import BytesIO
        import requests
        response = requests.get(url)
        img_data = Image.open(BytesIO(response.content))
        return img_data

if __name__ == "__main__":
    root = ThemedTk(theme="breeze")
    app = StudentPredictionApp(root)
    root.mainloop()
